<a href="https://colab.research.google.com/github/PavanReddy28/IR-Assignment/blob/main/IR_Assignment_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boolean Information Retrieval System

## Import Packages

In [97]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import os

from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [98]:
## Change this based on your file location
PARENT = '/content/gdrive/My Drive/3-2/IR/shakespeares-works_TXT_FolgerShakespeare'

## Load Data and Data Preprocessing

In [99]:
def loadFileInfo(parentDir):
    '''Loads name of files and vectorizes the file info'''
    folder = os.listdir(parentDir)
    folder.remove('__MACOSX')
    len(folder)
    file_index = dict()
    for i,j in zip(folder,range(1,len(folder)+1)):
        file_index[i] = j
    invertedFileIndex = {value:key for key,value in file_index.items()}
    return file_index, invertedFileIndex

file_index, invertedFileIndex = loadFileInfo(PARENT)

In [100]:
file_index.keys(), invertedFileIndex.keys()

(dict_keys(['henry-iv-part-1_TXT_FolgerShakespeare.txt', 'henry-vi-part-1_TXT_FolgerShakespeare.txt', 'henry-iv-part-2_TXT_FolgerShakespeare.txt', 'henry-vi-part-2_TXT_FolgerShakespeare.txt', 'henry-vi-part-3_TXT_FolgerShakespeare.txt', 'alls-well-that-ends-well_TXT_FolgerShakespeare.txt', 'as-you-like-it_TXT_FolgerShakespeare.txt', 'coriolanus_TXT_FolgerShakespeare.txt', 'much-ado-about-nothing_TXT_FolgerShakespeare.txt', 'the-comedy-of-errors_TXT_FolgerShakespeare.txt', 'henry-viii_TXT_FolgerShakespeare.txt', 'cymbeline_TXT_FolgerShakespeare.txt', 'julius-caesar_TXT_FolgerShakespeare.txt', 'king-john_TXT_FolgerShakespeare.txt', 'lucrece_TXT_FolgerShakespeare.txt', 'loves-labors-lost_TXT_FolgerShakespeare.txt', 'measure-for-measure_TXT_FolgerShakespeare.txt', 'king-lear_TXT_FolgerShakespeare.txt', 'the-merchant-of-venice_TXT_FolgerShakespeare.txt', 'macbeth_TXT_FolgerShakespeare.txt', 'a-midsummer-nights-dream_TXT_FolgerShakespeare.txt', 'othello_TXT_FolgerShakespeare.txt', 'richard-i

In [101]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import 	WordNetLemmatizer

def filterTokens(para):
    '''Toeknizes input strings, removes stop words, removes non alpha numeric characters'''
    stopWords = sorted(stopwords.words('english'))
    tokens = para.replace('\n', ' ').split(' ')
    filteredTokens = list(set([re.sub('[\W\_]','', word.lower()) for word in tokens if word.lower() not in stopWords]))
    t = filteredTokens.pop(0)
    return filteredTokens

def stemming(para):
    '''Stemming of words in input.'''
    stemmer  = PorterStemmer()
    return [stemmer.stem(word) for word in para]

def lemmatize(para):
    '''Lemmatize words in input.'''
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in para]

def genWordDict(file_index, normalize='stem'):
    '''Generates Word Dictionary contianing word indexes from each document. Includes preprocessing steps.'''
    nltk.download('stopwords')
    nltk.download('wordnet')
    Words = dict()
    fileInd = list(file_index.keys())
    print('Reading and Tokenizing files........')
    for i in tqdm(range(len(fileInd))):
        file = fileInd[i]
        f = open(os.path.join(PARENT, file), 'r')
        para = f.read()
        filteredTokens = filterTokens(para)
        normalized = None
        if normalize=='stem':
            normalized = stemming(filteredTokens)
        elif normalize=='lemmatize':
            normalized = lemmatize(filteredTokens)
        Words[file_index[file]] = normalized
    print('\nDone tokenizing and removeing stop words.')
    return Words
    
docWords = genWordDict(file_index)      # With Stemming
docWordsStem = docWords     # With Stemming
docWordsLem = genWordDict(file_index, normalize='lemmatize')    # With Lemmetization

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Reading and Tokenizing files........


100%|██████████| 42/42 [00:07<00:00,  5.54it/s]



Done tokenizing and removeing stop words.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Reading and Tokenizing files........


100%|██████████| 42/42 [00:04<00:00,  9.39it/s]


Done tokenizing and removeing stop words.


In [102]:
def genInverted(wordDict):
    '''Generates Inverted Word data and final sorted Inverted Index Data structure.'''
    Inv_word = dict()
    e_list = []
    Words = wordDict
    for i in Words.keys():
        Inv_word.update({k:list(e_list) for k in Words[i]})
    for i in Words.keys():
        for j in Words[i]:
            Inv_word[j].append(i)
    Inv_index = dict() 
    for word in Inv_word.keys():
        docs = list(set(Inv_word[word]))
        frequency = len(docs)
        docs.sort()
        Inv_index.update({word:[frequency,docs]})
    return Inv_word, Inv_index

InvWord, InvIndex = genInverted(docWords)

In [103]:
def viewInvIndex(InvIndex):
    '''Print Inverted Index.'''
    for k, (i,j) in enumerate(InvIndex.items()):
        print(str(i) +' - ' + str(j))

viewInvIndex(InvIndex)

Streaming output truncated to the last 5000 lines.
sendst - [1, [28]]
departest - [1, [28]]
49 - [1, [28]]
oersnow - [1, [28]]
1599 - [1, [28]]
74 - [1, [28]]
altr - [2, [28, 37]]
loan - [2, [28, 39]]
21 - [1, [28]]
116 - [1, [28]]
downraz - [1, [28]]
117 - [1, [28]]
clearer - [4, [28, 35, 37, 42]]
poesi - [4, [28, 29, 30, 33]]
113 - [1, [28]]
37 - [1, [28]]
perceivst - [1, [28]]
yore - [1, [28]]
38 - [1, [28]]
127 - [1, [28]]
bristli - [2, [28, 36]]
94 - [1, [28]]
30 - [1, [28]]
incertainti - [2, [28, 37]]
75 - [1, [28]]
gainer - [2, [28, 38]]
98 - [1, [28]]
livesuch - [1, [28]]
83 - [1, [28]]
20 - [1, [28]]
twilight - [1, [28]]
129 - [1, [28]]
hardest - [2, [28, 38]]
50 - [1, [28]]
59 - [1, [28]]
lovekindl - [1, [28]]
120 - [1, [28]]
85 - [1, [28]]
108 - [1, [28]]
139 - [1, [28]]
steepup - [1, [28]]
29 - [1, [28]]
debateth - [1, [28]]
newfound - [2, [28, 30]]
uplock - [1, [28]]
blessedfair - [1, [28]]
81 - [1, [28]]
idol - [7, [28, 30, 32, 36, 39, 40, 42]]
64 - [1, [28]]
123 - [1, [2

## Spelling Correction - Edit Distance

In [104]:
def editDistance(w1, w2):
    '''Returns the edit distance between two words.'''
    n1 = len(w1)
    n2 = len(w2)
    dp = [[0 for i in range(n1+1)] for j in range(2)]

    for i in range(n1+1):
        dp[0][i] = i
    
    for i in range(1, n2+1):
        for j in range(0, n1+1):
            if j==0:
                dp[i%2][j] = i
            elif w1[j-1]==w2[i-1]:
                dp[i%2][j] = dp[(i-1)%2][j-1]
            else:
                dp[i % 2][j] = 1 + min( dp[(i-1)%2][j], min(dp[i%2][j-1], dp[(i-1)%2][j-1]) )

    return dp[n2%2][n1]

editDistance('mac', 'macbeth')

4

In [105]:
def getCorrectWords(w1, InvIndex, top_k=10):
    '''Returns list of similar words based on edit distance along with their respective edit distances.'''
    editScores = dict()
    for word in InvIndex.keys():
        editScores[word] = editDistance(word, w1)
    editScoresList = list(editScores.items())
    editScoresList = sorted(editScoresList, key=(lambda x: x[1]))
    return editScoresList[:min(top_k, len(InvIndex.keys()))]

getCorrectWords('man', InvIndex, 5)

[('man', 0), ('mani', 1), ('can', 1), ('wan', 1), ('mean', 1)]

## List Intersection - tried to use skip pointers 


In [106]:
class Posting:
    def __init__(self,word,frequency,p_list):
        self.word = word
        self.frequency = frequency
        self.p_list = p_list

In [107]:
def list_intersection(l1,l2):
    intersection = []
    pointer_l1 = 0
    pointer_l2 = 0
    while(True):
        if (pointer_l1 == len(l1) or pointer_l2 == len(l2)):
            break
        if l1[pointer_l1] == l2[pointer_l2]:
            pointer_l1 += 1
            pointer_l2 += 1
            intersection.append(l1[pointer_l1-1])
        elif l1[pointer_l1] > l2[pointer_l2]:
            pointer_l2 += 1
        else:
            pointer_l1 += 1
    return intersection

In [108]:
l1 = InvIndex['idol'][1]
print(l1)

[28, 30, 32, 36, 39, 40, 42]


In [109]:
l2 = [3,4,5,12,16,18,40,43]
print(l2)

[3, 4, 5, 12, 16, 18, 40, 43]


In [110]:
list_intersection(l1,l2)

[40]

In [111]:
def skip_list_intersection(l1,l2):
    s1 = int(len(l1)**0.5)
    s2 = int(len(l2)**0.5)
    
    skip_intersection = []
    pointer_l1 = 0
    pointer_l2 = 0

    skip_nodes_l1 = [k for k in range(0,len(l1),s1)]
    skip_nodes_l2 = [k for k in range(0,len(l2),s2)]

    while(True):
        if (pointer_l1 == len(l1) or pointer_l2 == len(l2)):
            break
        if l1[pointer_l1] == l2[pointer_l2]:
            pointer_l1 += 1
            pointer_l2 += 1
            skip_intersection.append(l1[pointer_l1-1])
        elif l1[pointer_l1] > l2[pointer_l2]:
            if (pointer_l2 + s2 < len(l2) - 1):
                if (pointer_l2 in skip_nodes_l2):
                    if l2[pointer_l2 + s2] < l1[pointer_l1]:
                        pointer_l2 += s2
                    else:
                        pointer_l2 += 1
                else:
                    pointer_l2 += 1
            else:
                pointer_l2 += 1
            print(l2[pointer_l2-1])
        else:
            if (pointer_l1 in skip_nodes_l1):
                if l1[pointer_l1 + s1] < l2[pointer_l2]:
                    pointer_l1 += s1
                else:
                    pointer_l1 += 1
            else:
                pointer_l1 += 1
    return skip_intersection


In [112]:
def skip_list_intersection(l1,l2):
    s1 = int(len(l1)**0.5)
    s2 = int(len(l2)**0.5)
    
    skip_intersection = []
    pointer_l1 = 0
    pointer_l2 = 0

    skip_nodes_l1 = [k for k in range(0,len(l1),s1)]
    skip_nodes_l2 = [k for k in range(0,len(l2),s2)]

    while(True):
        if (pointer_l1 == len(l1) or pointer_l2 == len(l2)):
            break
        if l1[pointer_l1] == l2[pointer_l2]:
            pointer_l1 += 1
            pointer_l2 += 1
            skip_intersection.append(l1[pointer_l1-1])
        elif l1[pointer_l1] > l2[pointer_l2]:
            if (pointer_l2 + s2 < len(l2) - 1):
                if (pointer_l2 in skip_nodes_l2):
                    if l2[pointer_l2 + s2] < l1[pointer_l1]:
                        pointer_l2 += s2
                    else:
                        pointer_l2 += 1
                else:
                    pointer_l2 += 1
            else:
                pointer_l2 += 1
            print(l2[pointer_l2-1])
        else:

            #  if (pointer_l2 + s2 < len(l2) - 1):
            #     if (pointer_l2 in skip_nodes_l2):
            #         if l2[pointer_l2 + s2] < l1[pointer_l1]:
            #             pointer_l2 += s2
            #         else:
            #             pointer_l2 += 1
            #     else:
            #         pointer_l2 += 1
            #  else:
            #     pointer_l2 += 1
            # print(l2[pointer_l2-1])

            if pointer_l1+s1<len(l1)-1:
                if (pointer_l1 in skip_nodes_l1):
                    if l1[pointer_l1 + s1] < l2[pointer_l2]:
                        pointer_l1 += s1
                    else:
                        pointer_l1 += 1
                else:
                    pointer_l1 += 1
            else:
                pointer_l1 += 1
            print(l1[pointer_l1-1])
    return skip_intersection


In [113]:
l2 = [1,8,50,55,56,57,1000,1001,1005]
l1 = [1,2,3,4,5,7,8,50,91,92,93,94,95,100,1000,1001,1004,1005]

In [114]:
skip_list_intersection(l1,l2)

2
3
4
5
7
55
56
57
94
95
100
1004


[1, 8, 50, 1000, 1001, 1005]

In [115]:
for i in range(0,9,3):
    print(i)

0
3
6


## K Grams

In [116]:
def find_relevant_words(k_gram_index,wild_card_word,k):
  '''Find relevant words'''
  k_grams = []
  new_wild_card_word =  '$' + wild_card_word + '$'
  sub_words = new_wild_card_word.split('*')
  if '$' in sub_words:
    sub_words.remove('$')

  relevant_lists = []
  for word in sub_words:
    k_grams = getKGrams(word,k)
    if len(k_grams) == 0:
      k_grams = getKGrams(word,1)
  
    for k_gram in k_grams:
      relevant_lists.append(k_gram_index[k_gram])
  relevant_words = []
  end = 1
  print(len(relevant_lists))
  relevant_words = relevant_lists[0]
  while(end < len(relevant_lists)):
      relevant_words = np.intersect1d(relevant_words,relevant_lists[end])
      end += 1
  return relevant_words
  
 
def createKGramIndex(InvIndex,k):
  unique_words = list(InvIndex.keys())
  k_gram_index = dict()
  count = 0
  for word in unique_words:
    count += 1
    new_word = '$' + word + '$'
    k_grams = getKGrams(new_word,k)
    for k_gram in k_grams:
      if k_gram in k_gram_index.keys():
        k_gram_index[k_gram].append(word)
      else:
        k_gram_index[k_gram] = []
        k_gram_index[k_gram].append(word)
  
  for word in unique_words:
    count += 1
    new_word = '$' + word + '$'
    k_grams = getKGrams(new_word,1)
    for k_gram in k_grams:
      if k_gram in k_gram_index.keys():
        k_gram_index[k_gram].append(word)
      else:
        k_gram_index[k_gram] = []
        k_gram_index[k_gram].append(word)
  for key in k_gram_index.keys():
    k_gram_index[key].sort()
  return k_gram_index
      

In [117]:
k_gram_index = createKGramIndex(InvIndex,2)

In [118]:
def getKGrams(word,k):
  k_grams = []
  length = len(word)
  start = 0
  end = k
  while(end <= length):
    k_grams.append(word[start:end])
    start += 1
    end += 1
  return k_grams

getKGrams('love',2)

['lo', 'ov', 've']

## Query

In [119]:
def not_list(L,n):
    '''Code for finding out the not boolean operator'''
    not_list = []
    for i in range(1,n+1):
        if i not in L:
            not_list.append(i)
    return not_list

In [120]:
def union(l1,l2):
    '''Union of Two Lists'''
    return l1+l2

In [121]:
#Assuming the spellings are correct and the given the query we can do the folowing analysis
# query = 'brutus and man or caesar'

def get_relevant_docs(query,InvIndex,file_index):
    and_split = list()
    or_split = list()

    def get_or_split(query):
        l_and_splits = []
        l_or_splits = []

        or_split = query.split('or')
        for i,j in zip(or_split,range(0,len(or_split))):
            or_split[j] = i.strip()
        for i in or_split:
            if 'and' in i:
                l_and_splits.append(i)
            else:
                l_or_splits.append(i)
        return l_or_splits, l_and_splits
        
    l_or_splits, l_and_splits = get_or_split(query)

    def get_and_split(L):
        p_and_list= []
        for i in L:
            l = i.split('and')
            for j in l:
                p_and_list.append(j.strip())
        return p_and_list

    p_and_list = get_and_split(l_and_splits)


    #getting all the relevant documents from the corpus of documents
    all_docs_and = []

    for i in p_and_list:
        if 'not' not in i:
            try:
             
                all_docs_and.append(InvIndex[i][1])
            except:
                all_docs_and.append([])
        if 'not' in i:
            try:
             
                i = i.split(' ')[1]
                all_docs_and.append(not_list(InvIndex[i][1],len(file_index)))
            except:
                all_docs_and.append(not_list([],len(file_index)))


    all_docs_or = []
    for i in l_or_splits:
        if 'not' not in i:
            try:
           
                all_docs_or.append(InvIndex[i][1])
            except:
                all_docs_or.append([])
        if 'not' in i:
            try:
              
                i = i.split(' ')[1]
                all_docs_or.append(not_list(InvIndex[i][1],len(file_index)))
            except:
                all_docs_or.append(not_list([],len(file_index)))

    l = []
    almost_lists = []

    for i in range(0,len(all_docs_and)-1,2):
        l = np.intersect1d(all_docs_and[i],all_docs_and[i+1])
        almost_lists.append(l)

    final_lists = []
    for i in almost_lists:
        final_lists.extend(i)
    for i in all_docs_or:
        final_lists.extend(i)

    return list(set(final_lists))




In [129]:
def generate_query(query):
    all_words = query.split(' ')
    new_queries = []
    for i in all_words:
        if i not in ['and', 'or', 'not']:
            if '*' not in i:
                j = getCorrectWords(i,InvIndex, 1)[0][0]
                j = stemming([j])
                new_queries.append(j[0])

            else:
                j = find_relevant_words(k_gram_index,i,2)
                length = len(j)
                ll = int(np.log2(length+1))
                lll = j[0:ll]
                lll = stemming(lll)
                new_queries.append(list(lll))
        else:
            new_queries.append(i)
    return new_queries
    

In [133]:
def search_and_retrieve(query):
    '''Search and retrieve: Get all the docs for the given query post preprocessing'''
    q = generate_query(query)
    queries = []
    if type(q[-1]) == type([]):
        for i in q[-1]:
            nl = q[0:-1]
            nl.append(i)
            query = (' ').join(nl)
            queries.append(query)
    else:
        query = (' ').join(q)
        queries.append(query)
    print('\nQueries after preprocessing: ')
    for i, q in enumerate(queries):
        print('Query', i, ': ', q)
    print('\n')
    d = dict()
    for i in queries:
        d[i] = get_relevant_docs(i,InvIndex,file_index)
    print(d)

## INPUT QUERIES HERE

In [134]:
query = input('Provide Input Boolean Query: ')
search_and_retrieve(query)

Provide Input Boolean Query: shakespear and not brut*
4

Queries after preprocessing: 
Query 0 :  shakespear and not biddingbrutu
Query 1 :  shakespear and not brute
Query 2 :  shakespear and not brutethen


{'shakespear and not biddingbrutu': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], 'shakespear and not brute': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42], 'shakespear and not brutethen': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]}
